<a href="https://colab.research.google.com/github/yesyell/DA_School_1/blob/main/%ED%86%B5%EA%B3%84%EB%B6%84%EC%84%9D%20%EB%B0%A9%EB%B2%95%EB%A1%A0/AB_Test_%EC%8B%A4%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

가설 : 기존 웹사이트(A) 대비 신규 웹사이트(B)로 바꾸었을때, 고객 평균 체류시간이 늘어났다.

# AB 테스트 - 유저의 광고로 인한 방문 여부 테스트하기

In [ ]:
import numpy as np
import pandas as pd

# 데이터 생성
np.random.seed(0)
n = 100
group = np.random.choice(['A', 'B'], size=n, p=[0.5, 0.5])
time_on_site = np.where(group == 'A',
                        np.random.normal(loc=5, scale=1, size=n),
                        np.random.normal(loc=5.2, scale=1, size=n))
df = pd.DataFrame({'Group': group, 'TimeOnSite': time_on_site})

print("Data:")
df

Data:


,Group,TimeOnSite
0,B,4.846006
1,B,3.825049
2,B,4.556382
3,B,2.976597
4,A,6.488252
...,...,...
95,A,5.681595
96,B,5.693742
97,A,4.310450
98,B,3.169316


In [ ]:
from scipy import stats

# 데이터 나누기
Group_A = df[df['Group'] == 'A']
Group_B = df[df['Group'] == 'B']

Group_Test_A = Group_A['TimeOnSite']
Group_Test_B = Group_B['TimeOnSite']

In [ ]:
Group_A['TimeOnSite'].describe()

count    51.000000
mean      5.244532
std       1.015571
min       3.455229
25%       4.433061
50%       5.396007
75%       5.946950
max       7.383145
Name: TimeOnSite, dtype: float64

In [ ]:
Group_B['TimeOnSite'].describe()

count    49.000000
mean      4.963798
std       1.060401
min       2.427407
25%       4.460437
50%       5.067119
75%       5.579152
max       7.459309
Name: TimeOnSite, dtype: float64

In [ ]:
# 검정실시
stats.ttest_ind(Group_Test_A,
                Group_Test_B,
                equal_var=False) # 등분산성이 같지 않다

# 해당 옵션을 사용하면 Welch의 t-test를 사용하게 됩니다.
# 아직 여러분들이 통계학에 대한 이론적인 내용을 더 얻기에는 쉽지 않기 때문에 해당 내용을 가정하고 분석을 합니다.

# 귀무가설 A와 B가 같다.
# 대립가설 A와 B가 차이가 있다.
# 0.05 < 0.17 귀무가설을 기각할수 없어! -> 차이가 없다.

Ttest_indResult(statistic=1.3511318528865477, pvalue=0.17978594413747812)

In [ ]:
# 우리가 원하는것은 A(새로운 광고)가 B(기존 광고)보다 고객유입이 잘되는 것을 파악하기 위함
# Pvalue가 0.05보다 높으면 귀무가설을 기각할 수 없기 때문에 즉,새로운 광고가 더 낫다고 할 수 없음

* 가설검정은 각 그룹의 평균값으로 비교
* A가 평균 방문횟수가 더 많지만 유의미한 차이인지에 대한 의문
* 평균차이는 없었다. 라는 가설이 있으면 그것을 검정하게 됩니다. t-test(독립 t 검정)이라고 부릅니다.
* p-value는 평균의 차이가 있느냐 없느냐를 판단하는 척도 입니다. 전통적인 통계학은 p-value가 0.05보다 낮으면 그 결론을 기각한다 라고 말합니다.
* 현재 우리가 한 p-value값은 0.17이므로 통계적으로는 유의미하지 않습니다.
* AB테스트의 표면적인 결과는 0.3번만큼더 방문했지만 비즈니스적으로 광고를 추가적으로 해도 별차이가 없음을 보여줍니다.

In [ ]:
# equal_var = False 를 파라미터로 넣으면 사이파이의 패키지는 Welch의 t-test를 사용합니다.
# 이렇게 분산이 갖고 다름을 등분산성, 이분산성이라고 부릅니다.

# 통계학적으로는 데이터의 크기가 극단적으로 다르거나, 정규성과는 거리가 많이 멀거나,
# 분산이 달라짐에 따라 효과크기, 검정력 등의 다양한 요소가 영향을 주고 받기 때문에
# 등분산성을 어떻게 가정하고 t-test를 진행해야하는지 오늘날까지도 논란이 많습니다.

# 개인적으로, AB 테스트를 진행해야하는 실무자에게는 끊임 없는 학문적 논의보다는
# 빠른 의사결정과 자신의 데이터 환경에 맞는 방법론을 적절히 선택하는 게 중요하다고 생각합니다.
# 일단은 False로 두고 테스트를 하는 것을 추천드립니다.
# 실무에서 수집한 데이터의 분산이 같기는 드물 뿐더러 수집 데이터의 갯수가 같은 경우는 더욱 드물기 때문입니다.

# AB 테스트를 통해 수집한 데이터는 그 양이 지나치게 적지 않는 한
# 정규성 (극단적으로 크고 작은 값보다는 중간 정도의 값이 많은 성격) 정도는 보장되는 경우가 많은 것도
# 웰치의 t-test를 기본으로 사용하게 되는 이유 중 하나입니다.

In [ ]:
# 분산이 같을 경우

In [ ]:
group = ['B', 'B', 'B', 'B', 'B', 'B', 'G', 'G','G','G','G','G','G']
test = [8.8,8.4,7.9,8.7,9.1,9.6, 9.9, 9.0, 11.1, 9.6, 8.7, 10.4 ,9.5]
df = pd.DataFrame({'Group': group, 'test': test})
df

,Group,test
0,B,8.8
1,B,8.4
2,B,7.9
3,B,8.7
4,B,9.1
5,B,9.6
6,G,9.9
7,G,9.0
8,G,11.1
9,G,9.6


In [ ]:
# 분산이 동일한지 보려면 분산 동일성 검정을 수행해야합니다.
# stat 모듈의 stats.levene()으로 levene 검정을 수행합니다.

In [ ]:
from scipy import stats
drugB = df.loc[df.Group =='B', 'test']
drugG = df.loc[df.Group =='G', 'test']

In [ ]:
levene = stats.levene(drugB, drugG)
print(levene)

LeveneResult(statistic=0.5208519677996435, pvalue=0.485536773334305)


In [ ]:
# p값이 통상적인 기준 0.05, 또는 0.1보다도 크므로 두 그룹의 분산이 동일하다는 가정을 받아들입니다.
# 그렇지 않으면 분산이 다르다는 말입니다.
# 이런 경우에는 equal_var = True로 지정합니다.

In [ ]:
# 검정실시
stats.ttest_ind(drugB,
                drugG,
                equal_var=True) # 등분산성이 같음

# 두 그룹간에는 유의한 차이가 있다라고 말할 수 있습니다.

Ttest_indResult(statistic=-2.4764898139809586, pvalue=0.030764898866015765)

In [ ]:
# 이런 경우에는 다시 재수행해봅니다.
levene = stats.levene(Group_Test_A, Group_Test_B)
print(levene)

LeveneResult(statistic=0.15909024503434524, pvalue=0.6908632702377716)


In [ ]:
# 검정실시
stats.ttest_ind(Group_Test_A,
                Group_Test_B,
                equal_var=True) # 등분산성이 같음

# 원래 결과와는 큰 차이가 나지는 않습니다만 그럼에도 정확한 분석을 진행했다는 신빙성을 줄수 있습니다.

Ttest_indResult(statistic=1.3523108217489752, pvalue=0.1793882647737695)

# 고객의 클릭내용을 토대로 AB 테스트 하기

In [ ]:
# 카이제곱 독립성 검정을 사용할 예정
# 카이제곱은 기댓값으로부터 어떤 관찰값까지의 거리를 나타내는 측정치를 의미하는데
# 아까와 같이 평균의 수치를 검증하는 것이 아닌 범주별 건수를 비교할때 사용하는 방법입니다.

https://medium.com/@Aaron__Kim/a-b-%ED%85%8C%EC%8A%A4%ED%8A%B8-a-b-test-split-test-8269a1798762

In [ ]:
import numpy as np
import pandas as pd

# 데이터 생성
np.random.seed(0)
n = 200
group = np.random.choice(['A', 'B'], size=n, p=[0.5, 0.5])
click = np.where(group == 'A',
                 np.random.choice([0, 1], size=n, p=[0.6, 0.4]),
                 np.random.choice([0, 1], size=n, p=[0.55, 0.45]))
df = pd.DataFrame({'Group': group, 'Click': click})

print("Data:")
df

Data:


,Group,Click
0,B,0
1,B,1
2,B,0
3,B,1
4,A,0
...,...,...
195,A,1
196,A,0
197,A,0
198,A,0


In [ ]:
from scipy.stats import chi2_contingency

# 두 그룹의 데이터를 교차표(cross tabulation)로 변환
cross_tab = pd.crosstab(df['Group'], df['Click'])

# 카이제곱 검정 수행
chi2_stat, p_value, dof, expected = chi2_contingency(cross_tab)

print("Chi-square Test Result:")
print("Chi-square statistic:", chi2_stat)
print("p-value:", p_value)

Chi-square Test Result:
Chi-square statistic: 0.08572733762728141
p-value: 0.7696809054498869


In [ ]:
# 데이터간 비교 카이제곱 검정 결과는 p-value로 결정 -> 0.05보다 높으므로 차이점이 없음

In [ ]:
import pandas as pd
from scipy.stats import chi2_contingency

def click_abtest(a_click, total_a, b_click, total_b):

    click = [a_click, b_click] # A와 B의 클릭한 유저 수
    no_click = [total_a - a_click, total_b - b_click]  # A와 B의 클릭 안 한 유저 수
    cont_table = pd.DataFrame([click, no_click], columns=['A', 'B'], index=['click', 'no_click'])
    chi2, p_val, d_f, expected = chi2_contingency([click, no_click])

    print("카이제곱 통계량 :", format(chi2, '.5f'))
    print("pvalue :", format(p_val, '.5f'))

# A의 클릭수(13)와 A의 데이터 전체 개수(244), B의 클릭수(40)와 A의 데이터 전체 개수(250)를 정의한 클래스에 넣어줍니다.
click_abtest(13, 244, 40, 250)

카이제곱 통계량 : 13.59088
pvalue : 0.00023


In [ ]:
# p-value가 0.05보다 낮기 때문에 서로간의 차이가 있다고 확인!

In [ ]:
# 역시 p-value에 따라 차이를 확인해야합니다.
# 클릭을 했느냐 안했느냐 여부는 방문횟수나 시간의 양과는 다르게 특정값으로는 평균을 내는 방법이 다릅니다.
# 왜냐하면 0과 1로만 되어있는 데이터이기 때문입니다. 아무런 값이 없을수도 있습니다.
# 이러한 데이터를 보통 binary 데이터 또는 nominal 데이터라고도 합니다. (연속형이 아니기 때문에)

# 따라서 예제 1과 같은 다른 방법으로 접근해야하며, 카이제곱검정이라는 방법론을 사용합니다.

https://dewberry9.github.io/ab-test-frequentist-statistical-analysis